In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Prophet and optimization libraries
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import optuna
from optuna.samplers import TPESampler

# Baseline models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import json

# Set random seeds for reproducibility
np.random.seed(42)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

#==============================================================================
# TASK 1: DATA GENERATION
#==============================================================================

class SyntheticRetailDataGenerator:
    """
    Generates synthetic retail sales data with multiple seasonal patterns
    and holiday effects suitable for Prophet modeling.
    """

    def __init__(self, start_date='2020-01-01', periods=1095, freq='D'):
        """
        Parameters:
        -----------
        start_date : str
            Start date for the time series
        periods : int
            Number of data points (minimum 1095 for 3 years daily data)
        freq : str
            Frequency of data ('D' for daily)
        """
        self.start_date = start_date
        self.periods = periods
        self.freq = freq
        self.date_range = pd.date_range(start=start_date, periods=periods, freq=freq)

    def generate_trend(self, changepoints=[365, 730]):
        """Generate non-linear trend with changepoints"""
        t = np.arange(self.periods)
        trend = 1000 + 2 * t  # Base upward trend

        # Add changepoints
        for cp in changepoints:
            if cp < self.periods:
                trend[cp:] += 500 * (1 - np.exp(-0.003 * (t[cp:] - cp)))

        return trend

    def generate_yearly_seasonality(self):
        """Generate yearly seasonal pattern"""
        day_of_year = self.date_range.dayofyear
        yearly = 300 * np.sin(2 * np.pi * day_of_year / 365.25)
        return yearly

    def generate_weekly_seasonality(self):
        """Generate weekly seasonal pattern (lower sales on weekends)"""
        day_of_week = self.date_range.dayofweek
        weekly = np.zeros(self.periods)

        # Weekend effect (lower sales)
        weekend_mask = (day_of_week >= 5)
        weekly[weekend_mask] = -200

        # Mid-week peak
        midweek_mask = (day_of_week == 2) | (day_of_week == 3)
        weekly[midweek_mask] = 150

        return weekly

    def generate_holiday_effects(self):
        """Generate three major holiday effects"""
        holiday_effect = np.zeros(self.periods)

        for date in self.date_range:
            year = date.year

            # Holiday 1: New Year's Day (Jan 1) - 5 day effect
            new_year = pd.Timestamp(f'{year}-01-01')
            if abs((date - new_year).days) <= 2:
                holiday_effect[self.date_range.get_loc(date)] = 800

            # Holiday 2: Black Friday (4th Friday of November) - 7 day effect
            november_first = pd.Timestamp(f'{year}-11-01')
            black_friday = november_first + pd.DateOffset(days=(3 - november_first.dayofweek) % 7 + 21)
            if abs((date - black_friday).days) <= 3:
                holiday_effect[self.date_range.get_loc(date)] = 1200

            # Holiday 3: Christmas (Dec 25) - 10 day effect
            christmas = pd.Timestamp(f'{year}-12-25')
            days_to_christmas = (date - christmas).days
            if -7 <= days_to_christmas <= 2:
                # Ramp up before Christmas, drop after
                if days_to_christmas < 0:
                    holiday_effect[self.date_range.get_loc(date)] = 1000 * (1 + days_to_christmas / 7)
                else:
                    holiday_effect[self.date_range.get_loc(date)] = 600

        return holiday_effect

    def generate_noise(self, scale=100):
        """Generate random noise"""
        return np.random.normal(0, scale, self.periods)

    def generate_dataset(self):
        """Generate complete synthetic dataset"""
        trend = self.generate_trend()
        yearly = self.generate_yearly_seasonality()
        weekly = self.generate_weekly_seasonality()
        holidays = self.generate_holiday_effects()
        noise = self.generate_noise()

        # Combine all components
        sales = trend + yearly + weekly + holidays + noise

        # Ensure non-negative values
        sales = np.maximum(sales, 100)

        # Create DataFrame
        df = pd.DataFrame({
            'ds': self.date_range,
            'y': sales,
            'trend': trend,
            'yearly_seasonality': yearly,
            'weekly_seasonality': weekly,
            'holiday_effect': holidays
        })

        return df

    def create_holiday_dataframe(self):
        """Create holiday dataframe for Prophet"""
        holidays = []

        years = range(2020, 2024)

        for year in years:
            # New Year's Day
            holidays.append({
                'holiday': 'new_year',
                'ds': pd.Timestamp(f'{year}-01-01'),
                'lower_window': -2,
                'upper_window': 2
            })

            # Black Friday
            november_first = pd.Timestamp(f'{year}-11-01')
            black_friday = november_first + pd.DateOffset(days=(3 - november_first.dayofweek) % 7 + 21)
            holidays.append({
                'holiday': 'black_friday',
                'ds': black_friday,
                'lower_window': -3,
                'upper_window': 3
            })

            # Christmas
            holidays.append({
                'holiday': 'christmas',
                'ds': pd.Timestamp(f'{year}-12-25'),
                'lower_window': -7,
                'upper_window': 2
            })

        return pd.DataFrame(holidays)

#==============================================================================
# TASK 2: CROSS-VALIDATION STRATEGY
#==============================================================================

class TimeSeriesCrossValidator:
    """
    Implements robust time series cross-validation strategies
    including rolling window and expanding window methods.
    """

    def __init__(self, initial_train_size=730, horizon=90, period=90, method='rolling'):
        """
        Parameters:
        -----------
        initial_train_size : int
            Initial training period in days (2 years = 730 days)
        horizon : int
            Forecast horizon in days (90 days = ~3 months)
        period : int
            Spacing between cutoff dates in days
        method : str
            'rolling' or 'expanding' window
        """
        self.initial_train_size = initial_train_size
        self.horizon = horizon
        self.period = period
        self.method = method

    def split_data(self, df, test_size=90):
        """
        Split data into train and test sets

        Parameters:
        -----------
        df : pd.DataFrame
            Full dataset
        test_size : int
            Size of test set in days

        Returns:
        --------
        train_df, test_df : tuple
            Training and test dataframes
        """
        split_idx = len(df) - test_size
        train_df = df.iloc[:split_idx].copy()
        test_df = df.iloc[split_idx:].copy()

        return train_df, test_df

    def prophet_cross_validation(self, model, train_df):
        """
        Perform Prophet's built-in cross-validation

        Parameters:
        -----------
        model : Prophet
            Fitted Prophet model
        train_df : pd.DataFrame
            Training data

        Returns:
        --------
        cv_results : pd.DataFrame
            Cross-validation results with predictions and actuals
        """
        # Calculate initial training period
        initial_str = f'{self.initial_train_size} days'
        period_str = f'{self.period} days'
        horizon_str = f'{self.horizon} days'

        cv_results = cross_validation(
            model,
            initial=initial_str,
            period=period_str,
            horizon=horizon_str,
            parallel="processes"
        )

        return cv_results

    def calculate_cv_metrics(self, cv_results):
        """
        Calculate performance metrics from cross-validation results

        Parameters:
        -----------
        cv_results : pd.DataFrame
            Cross-validation results from Prophet

        Returns:
        --------
        metrics : dict
            Dictionary of performance metrics
        """
        # Calculate metrics
        perf_metrics = performance_metrics(cv_results, rolling_window=0.1)

        # Aggregate metrics
        metrics = {
            'rmse': perf_metrics['rmse'].mean(),
            'mape': perf_metrics['mape'].mean(),
            'mase': perf_metrics['mase'].mean() if 'mase' in perf_metrics.columns else None,
            'mae': perf_metrics['mae'].mean()
        }

        return metrics, perf_metrics

#==============================================================================
# TASK 3: HYPERPARAMETER OPTIMIZATION
#==============================================================================

class ProphetHyperparameterOptimizer:
    """
    Implements Bayesian hyperparameter optimization for Prophet using Optuna.
    Searches over key parameters including seasonality modes, changepoint
    prior scale, and seasonality strength.
    """

    def __init__(self, train_df, holidays_df, cv_strategy):
        """
        Parameters:
        -----------
        train_df : pd.DataFrame
            Training data
        holidays_df : pd.DataFrame
            Holiday definitions
        cv_strategy : TimeSeriesCrossValidator
            Cross-validation strategy
        """
        self.train_df = train_df
        self.holidays_df = holidays_df
        self.cv_strategy = cv_strategy
        self.best_params = None
        self.best_score = None
        self.study = None

    def objective(self, trial):
        """
        Optuna objective function for hyperparameter optimization

        Parameters:
        -----------
        trial : optuna.Trial
            Optuna trial object

        Returns:
        --------
        cv_rmse : float
            Cross-validated RMSE (to be minimized)
        """
        # Define hyperparameter search space
        params = {
            'changepoint_prior_scale': trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True),
            'seasonality_prior_scale': trial.suggest_float('seasonality_prior_scale', 0.01, 10, log=True),
            'holidays_prior_scale': trial.suggest_float('holidays_prior_scale', 0.01, 10, log=True),
            'seasonality_mode': trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative']),
            'changepoint_range': trial.suggest_float('changepoint_range', 0.8, 0.95),
            'yearly_seasonality': trial.suggest_categorical('yearly_seasonality', [10, 15, 20]),
            'weekly_seasonality': trial.suggest_categorical('weekly_seasonality', [3, 5, 7]),
        }

        try:
            # Initialize Prophet with suggested parameters
            model = Prophet(
                changepoint_prior_scale=params['changepoint_prior_scale'],
                seasonality_prior_scale=params['seasonality_prior_scale'],
                holidays_prior_scale=params['holidays_prior_scale'],
                seasonality_mode=params['seasonality_mode'],
                changepoint_range=params['changepoint_range'],
                yearly_seasonality=params['yearly_seasonality'],
                weekly_seasonality=params['weekly_seasonality'],
                daily_seasonality=False,
                holidays=self.holidays_df
            )

            # Fit model
            model.fit(self.train_df)

            # Perform cross-validation
            cv_results = self.cv_strategy.prophet_cross_validation(model, self.train_df)

            # Calculate RMSE
            cv_rmse = np.sqrt(mean_squared_error(cv_results['y'], cv_results['yhat']))

            return cv_rmse

        except Exception as e:
            print(f"Trial failed: {e}")
            return float('inf')

    def optimize(self, n_trials=50, timeout=3600):
        """
        Run Bayesian optimization

        Parameters:
        -----------
        n_trials : int
            Number of optimization trials
        timeout : int
            Maximum optimization time in seconds

        Returns:
        --------
        best_params : dict
            Best hyperparameter configuration
        """
        # Create Optuna study
        self.study = optuna.create_study(
            direction='minimize',
            sampler=TPESampler(seed=42)
        )

        # Run optimization
        print(f"Starting hyperparameter optimization with {n_trials} trials...")
        self.study.optimize(self.objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)

        # Store best parameters
        self.best_params = self.study.best_params
        self.best_score = self.study.best_value

        print(f"\nOptimization complete!")
        print(f"Best RMSE: {self.best_score:.2f}")
        print(f"Best parameters: {json.dumps(self.best_params, indent=2)}")

        return self.best_params

    def get_optimization_history(self):
        """
        Get optimization history for visualization

        Returns:
        --------
        history_df : pd.DataFrame
            Optimization history with trial number and objective values
        """
        trials_df = self.study.trials_dataframe()
        return trials_df

#==============================================================================
# TASK 4: MODEL TRAINING AND BASELINE COMPARISON
#==============================================================================

class ModelComparison:
    """
    Trains optimized Prophet model and compares against baseline models
    (ARIMA and Naive Seasonal Forecast) on held-out test set.
    """

    def __init__(self, train_df, test_df, holidays_df):
        """
        Parameters:
        -----------
        train_df : pd.DataFrame
            Training data
        test_df : pd.DataFrame
            Test data
        holidays_df : pd.DataFrame
            Holiday definitions
        """
        self.train_df = train_df
        self.test_df = test_df
        self.holidays_df = holidays_df
        self.models = {}
        self.predictions = {}
        self.metrics = {}

    def train_prophet(self, params):
        """
        Train Prophet model with optimized parameters

        Parameters:
        -----------
        params : dict
            Optimized hyperparameters

        Returns:
        --------
        model : Prophet
            Trained Prophet model
        """
        model = Prophet(
            changepoint_prior_scale=params['changepoint_prior_scale'],
            seasonality_prior_scale=params['seasonality_prior_scale'],
            holidays_prior_scale=params['holidays_prior_scale'],
            seasonality_mode=params['seasonality_mode'],
            changepoint_range=params['changepoint_range'],
            yearly_seasonality=params['yearly_seasonality'],
            weekly_seasonality=params['weekly_seasonality'],
            daily_seasonality=False,
            holidays=self.holidays_df
        )

        model.fit(self.train_df)
        self.models['prophet'] = model

        return model

    def predict_prophet(self):
        """Generate predictions using Prophet"""
        model = self.models['prophet']
        forecast = model.predict(self.test_df[['ds']])
        self.predictions['prophet'] = forecast['yhat'].values

        return self.predictions['prophet']

    def train_arima(self, order=(2, 1, 2), seasonal_order=(1, 1, 1, 7)):
        """
        Train ARIMA/SARIMA baseline model

        Parameters:
        -----------
        order : tuple
            ARIMA order (p, d, q)
        seasonal_order : tuple
            Seasonal order (P, D, Q, s)
        """
        try:
            from statsmodels.tsa.statespace.sarimax import SARIMAX

            model = SARIMAX(
                self.train_df['y'],
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            fitted_model = model.fit(disp=False, maxiter=200)
            self.models['arima'] = fitted_model

            # Generate predictions
            predictions = fitted_model.forecast(steps=len(self.test_df))
            self.predictions['arima'] = predictions.values

        except Exception as e:
            print(f"ARIMA training failed: {e}")
            # Fallback to simpler model
            self.predictions['arima'] = np.full(len(self.test_df), self.train_df['y'].mean())

    def naive_seasonal_forecast(self, season_length=7):
        """
        Create naive seasonal forecast baseline

        Parameters:
        -----------
        season_length : int
            Seasonal period (7 for weekly seasonality)
        """
        # Use last season's values as forecast
        last_season = self.train_df['y'].iloc[-season_length:].values

        # Repeat to match test set length
        n_repeats = int(np.ceil(len(self.test_df) / season_length))
        naive_pred = np.tile(last_season, n_repeats)[:len(self.test_df)]

        self.predictions['naive_seasonal'] = naive_pred

    def calculate_mase(self, y_true, y_pred, y_train):
        """
        Calculate Mean Absolute Scaled Error

        Parameters:
        -----------
        y_true : array
            Actual values
        y_pred : array
            Predicted values
        y_train : array
            Training data for scaling

        Returns:
        --------
        mase : float
            MASE metric
        """
        mae = mean_absolute_error(y_true, y_pred)

        # Calculate naive forecast MAE on training data
        naive_mae = np.mean(np.abs(np.diff(y_train)))

        if naive_mae == 0:
            return np.inf

        mase = mae / naive_mae
        return mase

    def evaluate_models(self):
        """
        Calculate performance metrics for all models

        Returns:
        --------
        metrics_df : pd.DataFrame
            DataFrame containing metrics for all models
        """
        y_true = self.test_df['y'].values
        y_train = self.train_df['y'].values

        metrics_list = []

        for model_name, y_pred in self.predictions.items():
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true, y_pred) * 100
            mase = self.calculate_mase(y_true, y_pred, y_train)

            metrics_list.append({
                'Model': model_name.upper(),
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape,
                'MASE': mase
            })

            self.metrics[model_name] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mase': mase
            }

        metrics_df = pd.DataFrame(metrics_list)
        return metrics_df

#==============================================================================
# TASK 5: ANALYSIS AND VISUALIZATION
#==============================================================================

class ResultsAnalyzer:
    """
    Provides comprehensive analysis and visualization of results including
    hyperparameter importance, model comparisons, and forecast visualizations.
    """

    def __init__(self, data_generator, optimizer, model_comparison):
        """
        Parameters:
        -----------
        data_generator : SyntheticRetailDataGenerator
            Data generator instance
        optimizer : ProphetHyperparameterOptimizer
            Hyperparameter optimizer instance
        model_comparison : ModelComparison
            Model comparison instance
        """
        self.data_generator = data_generator
        self.optimizer = optimizer
        self.model_comparison = model_comparison

    def plot_data_components(self, df):
        """Visualize the generated data and its components"""
        fig, axes = plt.subplots(5, 1, figsize=(14, 12))

        # Full time series
        axes[0].plot(df['ds'], df['y'], label='Sales', color='blue', linewidth=1)
        axes[0].set_title('Generated Retail Sales Data', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Sales')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Trend
        axes[1].plot(df['ds'], df['trend'], label='Trend', color='green', linewidth=1.5)
        axes[1].set_title('Trend Component', fontsize=12)
        axes[1].set_ylabel('Trend')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        # Yearly seasonality
        axes[2].plot(df['ds'], df['yearly_seasonality'], label='Yearly Seasonality', color='orange', linewidth=1)
        axes[2].set_title('Yearly Seasonal Component', fontsize=12)
        axes[2].set_ylabel('Yearly Effect')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)

        # Weekly seasonality
        axes[3].plot(df['ds'], df['weekly_seasonality'], label='Weekly Seasonality', color='purple', linewidth=1)
        axes[3].set_title('Weekly Seasonal Component', fontsize=12)
        axes[3].set_ylabel('Weekly Effect')
        axes[3].legend()
        axes[3].grid(True, alpha=0.3)

        # Holiday effects
        axes[4].plot(df['ds'], df['holiday_effect'], label='Holiday Effects', color='red', linewidth=1)
        axes[4].set_title('Holiday Effect Component', fontsize=12)
        axes[4].set_ylabel('Holiday Effect')
        axes[4].set_xlabel('Date')
        axes[4].legend()
        axes[4].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('data_components.png', dpi=300, bbox_inches='tight')
        plt.show()

        print("✓ Data components visualization saved as 'data_components.png'")

    def plot_optimization_history(self):
        """Visualize hyperparameter optimization progress"""
        trials_df = self.optimizer.get_optimization_history()

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # Optimization progress
        axes[0].plot(trials_df['number'], trials_df['value'], marker='o', markersize=4, alpha=0.6)
        axes[0].axhline(y=self.optimizer.best_score, color='r', linestyle='--',
                       label=f'Best: {self.optimizer.best_score:.2f}')
        axes[0].set_xlabel('Trial Number')
        axes[0].set_ylabel('CV RMSE')
        axes[0].set_title('Optimization Progress', fontweight='bold')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Parameter importance (if available)
        try:
            importance = optuna.importance.get_param_importances(self.optimizer.study)
            params = list(importance.keys())
            values = list(importance.values())

            axes[1].barh(params, values, color='steelblue')
            axes[1].set_xlabel('Importance')
            axes[1].set_title('Hyperparameter Importance', fontweight='bold')
            axes[1].grid(True, alpha=0.3, axis='x')
        except:
            axes[1].text(0.5, 0.5, 'Importance calculation unavailable',
                        ha='center', va='center', transform=axes[1].transAxes)

        plt.tight_layout()
        plt.savefig('optimization_history.png', dpi=300, bbox_inches='tight')
        plt.show()

        print("✓ Optimization history visualization saved as 'optimization_history.png'")

    def plot_model_comparison(self):
        """Visualize predictions from all models"""
        test_df = self.model_comparison.test_df

        fig, axes = plt.subplots(2, 1, figsize=(14, 10))

        # Full test period comparison
        axes[0].plot(test_df['ds'], test_df['y'], label='Actual', color='black',
                    linewidth=2, marker='o', markersize=3, alpha=0.7)

        colors = {'prophet': 'blue', 'arima': 'red', 'naive_seasonal': 'green'}
        for model_name, predictions in self.model_comparison.predictions.items():
            axes[0].plot(test_df['ds'], predictions, label=model_name.upper(),
                        color=colors[model_name], linewidth=1.5, alpha=0.7)

        axes[0].set_title('Model Predictions Comparison (Full Test Period)', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Sales')
        axes[0].legend(loc='upper left')
        axes[0].grid(True, alpha=0.3)

        # Zoomed view (first 30 days)
        zoom_days = 30
        axes[1].plot(test_df['ds'][:zoom_days], test_df['y'][:zoom_days],
                    label='Actual', color='black', linewidth=2, marker='o', markersize=4, alpha=0.7)

        for model_name, predictions in self.model_comparison.predictions.items():
            axes[1].plot(test_df['ds'][:zoom_days], predictions[:zoom_days],
                        label=model_name.upper(), color=colors[model_name],
                        linewidth=1.5, marker='s', markersize=3, alpha=0.7)

        axes[1].set_title(f'Model Predictions Comparison (First {zoom_days} Days)', fontsize=12, fontweight='bold')
        axes[1].set_xlabel('Date')
        axes[1].set_ylabel('Sales')
        axes[1].legend(loc='upper left')
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

        print("✓ Model comparison visualization saved as 'model_comparison.png'")

    def plot_metrics_comparison(self, metrics_df):
        """Visualize performance metrics comparison"""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        metrics = ['RMSE', 'MAE', 'MAPE', 'MASE']

        for idx, metric in enumerate(metrics):
            ax = axes[idx // 2, idx % 2]

            bars = ax.bar(metrics_df['Model'], metrics_df[metric], color=['blue', 'red', 'green'], alpha=0.7)
            ax.set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
            ax.set_ylabel(metric)
            ax.grid(True, alpha=0.3, axis='y')

            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}', ha='center', va='bottom', fontsize=10)

        plt.tight_layout()
        plt.savefig('metrics_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

        print("✓ Metrics comparison visualization saved as 'metrics_comparison.png'")

    def generate_report(self, metrics_df, full_df):
        """Generate comprehensive text report"""
        report = []
        report.append("="*80)
        report.append("ADVANCED TIME SERIES FORECASTING - COMPREHENSIVE REPORT")
        report.append("="*80)
        report.append("")

        # Dataset information
        report.append("1. DATASET INFORMATION")
        report.append("-" * 80)
        report.append(f"   • Total observations: {len(full_df)}")
        report.append(f"   • Date range: {full_df['ds'].min().date()} to {full_df['ds'].max().date()}")
        report.append(f"   • Training set size: {len(self.model_comparison.train_df)} days")
        report.append(f"   • Test set size: {len(self.model_comparison.test_df)} days")
        report.append(f"   • Mean sales: ${full_df['y'].mean():.2f}")
        report.append(f"   • Std deviation: ${full_df['y'].std():.2f}")
        report.append("")

        # Cross-validation strategy
        report.append("2. CROSS-VALIDATION STRATEGY")
        report.append("-" * 80)
        report.append("   Strategy: Time Series Split with Rolling Window")
        report.append(f"   • Initial training period: